# Modelo de Hodgkin–Huxley

### Refs.

[1] Eugene M. Izhikevich, *Dynamical Systems in Neuroscience: The Geometry of Excitability and Bursting*. Computational Neuroscience, The MIT Press, 1 edition (2006)

[2] https://github.com/amasky/hodgkin-huxley-model

[3] https://ocw.mit.edu/courses/brain-and-cognitive-sciences/9-40-introduction-to-neural-computation-spring-2018/index.htm

[4] https://www.youtube.com/playlist?list=PLUl4u3cNGP61I4aI5T6OaFfRK2gihjiMm

[5] https://www.mathcha.io/editor (app online para generar graficos con tikz)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def rk4(f,x,t,h,p):
    """
    Calcula un paso de integración del método de Runge Kutta orden 4.
    
    Argumentos de entrada:
    
        f : R^n -> R^n
        x = x(t) : R^n
        t = tiempo : R
        h = paso de tiempo : R
        p = parametros : R^q        
        
    Retorna aproximacion numérica de
    
        x(t+h) : R^n

    # Ejemplos:
    """    
    k1 = f(x,t,p)
    k2 = f(x+0.5*h*k1,t+0.5*h,p)
    k3 = f(x+0.5*h*k2,t+0.5*h,p)
    k4 = f(x+h*k3,t+h,p)
    return x+h*(k1+2.0*k2+2.0*k3+k4)/6.0

In [ ]:
def integrador_ode(m,f,x0,a,b,k,p):
    """
    Integra numéricamente la ODE
    
        dx/dt = f(x,t)
        
    sobre el intervalo t:[a,b] usando k pasos de integración y el método m, bajo condicion inicial x(a)=x0.
    No es necesario que a<b.
    
    Argumentos de entrada:
    
        m = metodo de integracion (ej. euler, rk2, etc.)
        f : R^n -> R^n
        x0 = condicion inicial : R
        a = tiempo inicial : R
        b = tiempo final : R
        k = num. pasos de integracion : N
        p = parametros : R^q
    
    Retorna:
    
        t : R^{k+1} , t_j = a+j*h para j=0,1,...,k
        x : R^{k+1,n} , x_ij = x_i(t_j) para i=0,1,...,n-1 y j=0,1,...,k
        
    donde a+k*dt = b.
    """  
    assert k>0
    n = len(x0)
    h = (b-a)/k
    x = np.zeros((n,k+1))
    t = np.zeros(k+1)
    x[:,0] = x0
    t[0] = a
    for j in range(k):
        t[j+1] = t[j] + h
        x[:,j+1] = m(f,x[:,j],t[j],h,p)
    return t,x

## Teoría

### Las neuronas son capacitores

Según el modelo de Hodgkin y Huxley, a primera vista las neuronas funcionan como capacitores evidenciando una diferencia de potencial eléctrico entre el exterior y el interior debido a las diferentes concentraciones de cargas iónicas que pueden existir entre estas dos regiones.
En el esquema, se utilizan agujas muy finas para inyectar una corriente iónica $i$ al interior de una neurona sumergida en una solución salina conductora.

![](esquema1.png "")

Recordemos que un capacitor sujeto a una diferencia de potencial $v:=v_A-v_B$ adquiere una carga $q=cv$.
La carga crece según $\dot{q}=i$.

![](circuito1.png "")

### Las neuronas son capacitores con fugas

Si miramos un poco más en detalle, veremos que la membrana de las neuronas tienen canales por donde pueden circular iones.
Podemos pensar a los canales como agujeritos en la membrana por donde la carga se fuga.
En otras palabras, podemos pensar a los canales como a una resistencia conectada en paralelo al capacitor.

![](circuito2.png "")

La corriente $i_g=g(v_A-v_B)=gv$.

### Las neuronas tienen baterias

Los canales pueden ser selectivos.
Por ejemplo, algunos canales sólo dejan pasar los iones $K^+$.
Por ende, si inyectemos al interior de la neurona una mezcla de iones $K^+$ y $Cl^-$, de manera que la carga total se encuentre equilibrada, los iónes $K^+$ comenzarán a difundir desde el interior hacia el exterior a travéz de los canales selectivos, mientras que los de $Cl^-$ se quedarán en el interior.
Esto generará un creciente excedente de cargas negativas en el interior y positivas en el exterior, hasta alcanzar un estado de equilibrio en donde la corriente de iones $K^+$ provocada por la difusión se contrabalancea a una corriente en dirección opuesta generada por la diferencia de potencial emergente.

![](baterias.png "")

Este potencial se llama potencial de Nernst o de reversión y se lo incorpora al circuito que modela la neurona como una batería conectada en serie a la resistencia asociada a cada canal

![](circuito3.png "")

Ahora, la corriente $i_g=g(v_A-v_C)=g(v_A-(v_B+e_g))=g(v_A-v_B-e_g)=g(v-e_g)=:g(v-v_g)$, donde en la última iguadad reconocemos la definición del potencial de Nernst $v_g$.

Si la batería se encuentra en sentido inverso

![](circuito4.png "")

entonces $i_g=g(v_A-v_C)=g(v_A-(v_B-e_g))=g(v_A-v_B+e_g)=g(v+e_g)=:g(v-v_g)$, por lo que el potencial de Nernst resulta negativo, $v_g=-e_g$.


### Algunos canales forman reóstatos (resistencias de conductancia variable)

Algunos canales tienen compuertas que se abren o cierran, permitiendo o impidiendo el paso de iones, respectivamente.

![](compuertas.png "")

La conductividad asociada a estos canales, depende de la fracción de compuertas abiertas en cada tipo de canal.
Según el modelo, la conductancia total asociada a los canales de $Na$ se aproxima por

$$g_{Na} = \bar{g}_{Na}p_{Na}$$

donde $\bar{g}_{Na}$ es la conductancia de $Na$ máxima posible y 

$$p_{Na} = m^3h$$

es la fracción de canales de $Na$ abiertos.
Aquí $m$ y $h$ son la fracciones de compuertas abiertas de tipos $m$ y $h$, respectivamente.
Los canales de $Na$ poseen 3 compuertas de tipo $m$ y 1 de tipo $h$, de allí los exponentes.

De manera similar, la conductancia asociada a los canales de $K$ se aproxima por

$$g_{K} = \bar{g}_{K}p_{K}$$

donde $\bar{g}_{K}$ es la conductancia de $K$ máxima posible,

$$p_{K} = n^4$$

es la fracción de canales de $K$ abiertos y $n$ es la fracción de compuertas abiertas de tipo-$n$.
Cada canal de $K$ posee 4 compuertas de tipo $n$.

Estas aproximaciones asumen que las compuertas abren o cierran de manera independiente, dependiendo de la diferencia de potencial de membrana existente $v$.
Más precisamente, las fracciones de compuertas abiertas de cada tipo satisfacen

\begin{eqnarray}
\dot{n}&=&\alpha_n(v)(1-n)-\beta_n(v) n\\
\dot{m}&=&\alpha_m(v)(1-m)-\beta_m(v) m\\
\dot{h}&=&\alpha_h(v)(1-h)-\beta_h(v) h
\end{eqnarray}

donde

\begin{eqnarray}
\alpha_h(v)&=&\frac{0.07}{ms\,mV}\,\exp\bigg(-\frac{v}{20\,mV}\bigg)\\
\alpha_m(v)&=&\frac{0.1}{ms\,mV}\frac{25\,mV-v}{\exp\big(\frac{25\,mV-v}{10\,mV}\big)-1}\\
\alpha_n(v)&=&\frac{0.01}{ms\,mV}\frac{10\,mV-v}{\exp\big(\frac{10\,mV-v}{10\,mV}\big)-1}\\
\end{eqnarray}

representan las tasas a las cuales las compuertas cerradas de cada tipo abren y

\begin{eqnarray}
\alpha_h(v)&=&\frac{0.07}{ms\,mV}\,\exp\bigg(-\frac{v}{20\,mV}\bigg)\\
\alpha_m(v)&=&\frac{0.1}{ms\,mV}\frac{25\,mV-v}{\exp\big(\frac{25\,mV-v}{10\,mV}\big)-1}\\
\alpha_n(v)&=&\frac{0.01}{ms\,mV}\frac{10\,mV-v}{\exp\big(\frac{10\,mV-v}{10\,mV}\big)-1}\\
\beta_h(v)&=&\frac{1}{ms}\frac{1}{\exp\big(\frac{30\,mV-v}{10\,mV}\big)+1}\\
\beta_m(v)&=&\frac{4}{ms}\exp\bigg(-\frac{v}{18\,mV}\bigg)\\
\beta_n(v)&=&\frac{0.125}{ms}\exp\bigg(-\frac{v}{80\,mV}\bigg)
\end{eqnarray}

representan las tasas a las cuales las compuertas abiertas de cada tipo cierran, respectivamente.

En resumen, al modelo de neurona de Hodgkin y Huxley se lo puede representar por el circuito

![](circuito5.png "")

en donde reconocemos la existencia de 3 canales iónicos: uno de pérdida (leak) de conductancia constante $g_l$, potencial de Nernst $v_l=e_l$ y por el que fluye una corriente $i_l$, uno de $Na$ de conductancia variable $g_{Na}$, potencial de Nernst $v_{Na}=e_{Na}$ y por el que fluye una corriente $i_{Na}$, y uno de $K$ de conductancia $g_K$, potencial de Nernst $v_K=-e_K$ y por el que fluye una corriente $i_K$.

Aplicando la ley de Kirchoff para corrientes, vemos que

$$i=i_{Na}+i_K+i_l+i_c$$

Es decir, la corriente $i$ inyectada al interior de la neurona se divide en dos partes.
Por un lado, en la corriente $i_m=i_{Na}+i_K+i_l$ que fluye a travéz de los canales en la membrana,
y por otro lado, en la corriente $i_c$ que carga el capacitor.

Remplazando, obtenemos una ODE para el potencial de membrana

\begin{eqnarray}
i_c&=&i-i_{Na}+i_K+i_l\\
c\dot{v}&=&i-g_{Na}(v-v_{Na})-g_{K}(v-v_{K})-g_{l}(v-v_{l})\\
\dot{v}&=&c^{-1}(i-\bar{g}_{Na}m^3h(v-v_{Na})-\bar{g}_{K}n^3(v-v_{K})-g_{l}(v-v_{l}))
\end{eqnarray}

que junto con las ODE anteriormente mencionadas para $n$, $m$ y $h$, determinan el sistema de ODEs del modelo de Hodgkin y Huxley.

### Comportamiento cualitativo

Durante el proceso de disparo, diferentes compuertas se abren y cierran a diferentes tiempos.
El siguiente esquema resume el proceso en 4 etapas

![](channels.png "")

Estimaciones de los tiempos carácterísticos y valores de equilibrio del comportamiento de las compuertas observado en las diferentes etapas se pueden obtener de reescribir las ODEs para $n$, $m$ y $h$ de la forma

\begin{eqnarray}
\dot{n}&=&\tau_n^{-1}(v)(n_{\infty}(v)-n)\\
\dot{m}&=&\tau_m^{-1}(v)(m_{\infty}(v)-m)\\
\dot{h}&=&\tau_h^{-1}(v)(h_{\infty}(v)-h)
\end{eqnarray}

donde 

\begin{eqnarray}
\tau_n(v)&=&(\alpha_n(v)+\beta_n(v))^{-1}\\
\tau_m(v)&=&(\alpha_m(v)+\beta_m(v))^{-1}\\
\tau_h(v)&=&(\alpha_h(v)+\beta_h(v))^{-1}
\end{eqnarray}

y

\begin{eqnarray}
n_{\infty}(v)&=&\tau_n(v)\alpha_n(v)\\
m_{\infty}(v)&=&\tau_m(v)\alpha_m(v)\\
h_{\infty}(v)&=&\tau_h(v)\alpha_h(v)
\end{eqnarray}

Finalmente, listamos los valores de los parámetros del modelo (ver libro de Izhikevich)

* $c=1\,\mu F/cm^2$ : capacitancia de membrana
* $\bar{g}_{Na}=120\,mS/cm^2$ : conductancia máxima de Na
* $\bar{g}_K=36\,mS/cm^2$ : conductancia máxima de K
* $g_l=0.3\,mS/cm^2$ : conductancia máxima de perdida
* $v_{Na}=120\,mV$ : potencial de reversión de la corriente de Na
* $v_K=-12\,mV$ : potencial de reversión de la corriente de K
* $v_l=10.6\,mV$ : potencial de reversión de la corriente de perdida
* $i(t)\sim 10\,\mu A/cm^2$ : corriente de entrada al tiempo $t$

Aclaración, los valores de los potenciales de Nernst se toman en referencia al potencial de equilibrio definido en $v\approx 0$.

In [ ]:
# Definamos las tasas de apertura y cierre de las diferentes compuertas.
def alpha_n(v):
    return 0.01*(10.0-v)/(np.exp((10.0-v)/10.0)-1.0)
def alpha_m(v):
    return 0.1*(25.0-v)/(np.exp((25.0-v)/10.0)-1.0)
def alpha_h(v):
    return 0.07*np.exp(-v/20.0)
def beta_n(v): 
    return 0.125*np.exp(-v/80.0)  
def beta_m(v):
    return 4.0*np.exp(-v/18.0)
def beta_h(v):
    return 1.0/(np.exp((30.0-v)/10.0)+1.0)

In [ ]:
# ...y los correspondientes tiempos característicos y valores de equilibrio.
def tau_n(v): 
    return 1.0/(alpha_n(v)+beta_n(v))
def tau_m(v): 
    return 1.0/(alpha_m(v)+beta_m(v))
def tau_h(v): 
    return 1.0/(alpha_h(v)+beta_h(v))
def n_inf(v): 
    return alpha_n(v)*tau_n(v)
def m_inf(v): 
    return alpha_m(v)*tau_m(v)
def h_inf(v): 
    return alpha_h(v)*tau_h(v)

In [ ]:
# Graficamos los valores de equilibrio de las distintas fracciones de compuertas abiertas.
plt.xlabel('$v$ (mV)')
#plt.ylabel('$n,m,h$')
plt.xlim(-50,120)
plt.ylim(-0.1,1.1)
v=np.linspace(-50,120,1000)
plt.plot(v,0*v,label="",linestyle='--',c='gray')
plt.plot(v,0*v+1,label="",linestyle='--',c='gray')
plt.plot(v,np.vectorize(n_inf)(v),label="$n_{\\infty}$",c='orange')
plt.plot(v,np.vectorize(m_inf)(v),label="$m_{\\infty}$",c='green')
plt.plot(v,np.vectorize(h_inf)(v),label="$h_{\\infty}$",c='cyan')
plt.title('Valores de equilibrio de las distintas fracciones de compuertas abiertas')
plt.legend()

In [ ]:
# Graficamos los tiempos característicos
plt.xlabel('$v$ (mV)')
plt.ylabel('$\\tau$ (ms)')
plt.xlim(-50,120)
#plt.ylim(-0.1,1.1)
v=np.linspace(-50,120,1000)
plt.plot(v,0*v,label="",linestyle='--',c='gray')
plt.plot(v,np.vectorize(tau_n)(v),label="$\\tau_n$",c='orange')
plt.plot(v,np.vectorize(tau_m)(v),label="$\\tau_m$",c='green')
plt.plot(v,np.vectorize(tau_h)(v),label="$\\tau_h$",c='cyan')
plt.title('Tiempos característicos')
plt.legend()

In [ ]:
# Implementamos el lado derecho de la ODE.

def ina(x,p):
    v = x[0]
    m = x[2]
    h = x[3]    
    gna = p[1]
    vna = p[4]
    return gna*(v-vna)*m**3*h

def ik(x,p):
    v = x[0]
    n = x[1]
    gk = p[2]
    vk = p[5]
    return gk*(v-vk)*n**4

def il(x,p):
    v = x[0]    
    gl = p[3]
    vl = p[6]
    return gl*(v-vl)

def f(x,t,p):
    """
    x[0] = v(t)    : potencial de membrana
    x[1] = n(t)    : activacion de K
    x[2] = m(t)    : activacion de Na
    x[3] = h(t)    : inactivacion de Na 
    t              : tiempo (ms)
    p[0] = c       : capacitancia de membrana (microF/cm**2)
    p[1] = gna     : conductancia máxima de Na (mS/cm**2)
    p[2] = gk      : conductancia máxima de K (mS/cm**2)    
    p[3] = gl      : conductancia máxima de perdida (mS/cm**2)
    p[4] = vna     : potencial de Nernst de K (mV)
    p[5] = vk      : potencial de Nernst Na (mV)
    p[6] = vl      : potencial de Nernst de la corriente de pérdida (mV)
    p[7] = t->i(t) : función corriente de entrada (microA/cm**2)
    Retorna [dv/dt,dn/dt,dm/dt,dh/dt]
    """
    v = x[0]
    n = x[1]
    m = x[2]
    h = x[3]
    c   = p[0]
    gna = p[1]
    gk  = p[2]    
    gl  = p[3]
    vna = p[4]
    vk  = p[5]    
    vl  = p[6]
    i   = p[7]
    return np.array([
        (i(t)-ina(x,p)-ik(x,p)-il(x,p))/c,
        alpha_n(v)*(1.0-n)-beta_n(v)*n,
        alpha_m(v)*(1.0-m)-beta_m(v)*m,
        alpha_h(v)*(1.0-h)-beta_h(v)*h,
    ])

In [ ]:
# Definimos parámetros
c = 1.        
gna = 120.
gk = 36.      
gl = 0.3      
vna = 120.0   
vk = -12.0
vl = 10.6     

In [ ]:
# Definimos condiciones iniciales preliminares.
v0 = 0.0
n0 = 0.0
m0 = 0.0
h0 = 0.0
x0 = np.array([v0,h0,m0,n0])

In [ ]:
# Definimos una corriente de membrana
def i(t):
    if t>=2.0 and t<=2.5:
        return 10.0
    elif t>=10.0 and t<=10.5:
        return 30.0
    else:
        return 0.0

In [ ]:
# Graficamos corriente de membrana.
plt.xlabel('$t$ (ms)')
plt.ylabel('$i$ $\mathrm{(\mu/cm^2)}$')
plt.xlim(-1,20)
plt.ylim(-2,40)
t = np.linspace(-1,30,1000)
plt.plot(t,np.vectorize(i)(t),c='r')
plt.title('Corriente de membrana')

Integramos

In [ ]:
# Integramos unos cuantos ms a corriente i=0 para determinar el punto de equilibrio del sistema.
v0 = 0.0 # membrane potential (mV)
n0 = 0.0 # K channel activation (dimensionless)
m0 = 0.0 # Na channel activation (dimensionless)
h0 = 0.0 # Na channel inactivation (dimensionless)
tini = 0 # ms
tend = 500 # ms
h = 0.01 # ms
k = int((tend-tini)/h) 
p = [c,gna,gk,gl,vna,vk,vl,lambda t:0]
x0 = np.array([v0,h0,m0,n0])
t,x = integrador_ode(rk4,f,x0,tini,tend,k,p)
x0=x[:,-1]

In [ ]:
ina(x0,p),ik(x0,p),il(x0,p)

In [ ]:
x0[0],x0[1],x0[2],x0[3]

In [ ]:
# Graficamos las corrientes
# Graficamos el potencial de membrana
plt.xlabel('$t$ (ms)')
plt.ylabel('(microA)')
plt.xlim(0,50)
plt.ylim(-5,5)
plt.plot(t,0*t,linestyle='--',c='gray')
plt.plot(t,[ina(x[:,j],p) for j in range(len(t))],c='r',label='i_Na')
plt.plot(t,[ik(x[:,j],p) for j in range(len(t))],c='g',label='i_K')
plt.plot(t,[il(x[:,j],p) for j in range(len(t))],c='b',label='i_l')
plt.legend()
plt.title('Potencial de membrana')

In [ ]:
# Integramos desde el equilibrio con la corriente dependiente del tiempo definida anteriormente.
tini = 0 # ms
tend = 20 # ms
h = 0.01 # ms
k = int((tend-tini)/h) 
p = [c,gna,gk,gl,vna,vk,vl,i]
t,x = integrador_ode(rk4,f,x0,tini,tend,k,p)

In [ ]:
# Graficamos el potencial de membrana
plt.xlabel('$t$ (ms)')
plt.ylabel('$v$ (mV)')
plt.xlim(0,20)
plt.ylim(-20,120)
plt.plot(t,0*t,linestyle='--',c='gray')
plt.plot(t,x[0],c='b')
plt.title('Potencial de membrana')

In [ ]:
# Graficamos el nivel de activación de cada canal: n, m y h.
plt.xlabel('$t$ (ms)')
plt.xlim(0,20)
plt.ylim(-0.1,1.1)
plt.plot(t,0*t,label="",linestyle='--',c='gray')
plt.plot(t,0*t+1,label="",linestyle='--',c='gray')
plt.plot(t,x[1],label="n",c='orange')
plt.plot(t,x[2],label="m",c='green')
plt.plot(t,x[3],label="h",c='cyan')
plt.title('Variables de activación')
plt.legend()

## Ráfaga

In [ ]:
# Definimos la corriente de membrana
def i(t):
    if t>=5.0:
        return 10.0
    else:
        return 0.0

In [ ]:
plt.xlabel('$t$ (ms)')
plt.ylabel('$i$ $\mathrm{(\mu/cm^2)}$')
plt.xlim(-1,100)
t = np.linspace(-1,100,1000)
plt.plot(t,np.vectorize(i)(t),c='r')
plt.title('Corriente de membrana')

In [ ]:
# Integramos desde el equilibrio
tini = 0 # ms
tend = 100 # ms
h = 0.01 # ms
k = int((tend-tini)/h) 
p = [c,gna,gk,gl,vna,vk,vl,i]
t,x = integrador_ode(rk4,f,x0,tini,tend,k,p)

In [ ]:
# Graficamos potencial de membrana
plt.xlabel('$t$ (ms)')
plt.ylabel('$v$ (mV)')
plt.xlim(0,100)
plt.ylim(-20,120)
plt.plot(t,0*t,linestyle='--',c='gray')
plt.plot(t,x[0],c='b')
plt.title('Potencial de membrana')

In [ ]:
# Graficamos activacion de canales
plt.xlabel('$t$ (ms)')
plt.xlim(0,100)
plt.ylim(-0.1,1.1)
plt.plot(t,0*t,label="",linestyle='--',c='gray')
plt.plot(t,0*t+1,label="",linestyle='--',c='gray')
plt.plot(t,x[1],label="n",c='orange')
plt.plot(t,x[2],label="m",c='green')
plt.plot(t,x[3],label="h",c='cyan')
plt.title('Variables de activación')
plt.legend()

## Período refractario

In [ ]:
def i(t):
    for n in range(1,10):
        if t>=10*n and t<=10*n+2:
            return 10.0
    return 0.0

In [ ]:
plt.xlabel('$t$ (ms)')
plt.ylabel('$i$ $\mathrm{(\mu/cm^2)}$')
plt.xlim(-1,50)
t = np.linspace(-1,100,1000)
plt.plot(t,np.vectorize(i)(t),c='r')
plt.title('Corriente de membrana')

In [ ]:
tini = 0 # ms
tend = 50 # ms
h = 0.01 # ms
k = int((tend-tini)/h) 
p = [c,gna,gk,gl,vna,vk,vl,i]
t,x = integrador_ode(rk4,f,x0,tini,tend,k,p)

In [ ]:
plt.xlabel('$t$ (ms)')
plt.ylabel('$v$ (mV)')
plt.xlim(0,50)
plt.ylim(-20,120)
plt.plot(t,0*t,linestyle='--',c='gray')
plt.plot(t,x[0],c='b')
plt.title('Potencial de membrana')

In [ ]:
plt.xlabel('$t$ (ms)')
plt.xlim(0,50)
plt.ylim(-0.1,1.1)
plt.plot(t,0*t,label="",linestyle='--',c='gray')
plt.plot(t,0*t+1,label="",linestyle='--',c='gray')
plt.plot(t,x[1],label="n",c='orange')
plt.plot(t,x[2],label="m",c='green')
plt.plot(t,x[3],label="h",c='cyan')
plt.title('Variables de activación')
plt.legend()

## Resistencia al ruido

In [ ]:
# i0 : nivel de ruido determinado por la desviación estandard de una distribucion normal centrada en 0.
# Importante, i0 ~ sqrt{h}, el paso de integración.
i0=50
def i(t):
    return i0*np.random.normal()

In [ ]:
plt.xlabel('$t$ (ms)')
plt.ylabel('$i$ $\mathrm{(\mu/cm^2)}$')
plt.xlim(-1,100)
t = np.linspace(-1,100,1000)
plt.plot(t,np.vectorize(i)(t),c='r')
plt.title('Corriente de membrana')

In [ ]:
tini = 0 # ms
tend = 100 # ms
h = 0.01 # ms
k = int((tend-tini)/h) 
p = [c,gna,gk,gl,vna,vk,vl,i]
t,x = integrador_ode(rk4,f,x0,tini,tend,k,p)

In [ ]:
plt.xlabel('$t$ (ms)')
plt.ylabel('$v$ (mV)')
plt.xlim(0,100)
plt.ylim(-20,120)
plt.plot(t,0*t,linestyle='--',c='gray')
plt.plot(t,x[0],c='b')
plt.title('Potencial de membrana')

In [ ]:
plt.xlabel('$t$ (ms)')
plt.xlim(0,100)
plt.ylim(-0.1,1.1)
plt.plot(t,0*t,label="",linestyle='--',c='gray')
plt.plot(t,0*t+1,label="",linestyle='--',c='gray')
plt.plot(t,x[1],label="n",c='orange')
plt.plot(t,x[2],label="m",c='green')
plt.plot(t,x[3],label="h",c='cyan')
plt.title('Variables de activación')
plt.legend()